# **멜론 인기 차트 크롤링하기 (BeautifulSoup/Selenium)**
마지막 수정일 : 2021년 4월 16일

## **1. BeautifulSoup을 이용한 크롤링**

- BeautifulSoup은 파이썬 웹 크롤링에 가장 널리 사용되는 라이브러리로, HTML 문서를 탐색해서 원하는 데이터를 쉽게 추출해 낼 수 있다.

- Beautifulsoup을 사용해 크롤링을 하는 경우, 서버에서 봇으로 인지하고 차단하는 경우가 발생할 수 있다. 이를 방지하기 위해 headers를 입력해준다. headers 정보에는 {'User-Agent' : '유저정보'} 를 넣어줘야 하는데, 이 '유저정보'는 아래의 사이트에서 얻을 수 있다.  
 http://www.useragentstring.com/ 
 
- HTML에서 필요한 데이터를 가져오기 위해 다양한 메소드를 사용할 수 있다. 가장 기본적인 메소드는 아래와 같고, 자세한 내용은 [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)에서 확인할 수 있다. 

      *   find()
      *   find_all()
      *   select_one()
      *   select()
      *   get_text()

### **1-1. 필요한 라이브러리 불러오기**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### **1-2. 헤더 설정 및 웹 페이지 요청하기**

In [2]:
hdr = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64)' 
                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36')}

response = requests.get('https://www.melon.com/chart/index.htm', headers = hdr) 
html = response.text
bs = BeautifulSoup(html, 'html.parser')

### **1-3. 데이터 가져오기**

In [3]:
TITLE = [] # 노래 제목
title = bs.find_all('div', {'class':'ellipsis rank01'})
for i in title:
    TITLE.append(i.find('a').get_text())

ARTIST = [] # 아티스트명
artist = bs.find_all('div', {'class':'ellipsis rank02'})
for i in artist:
    ARTIST.append(i.find('a').get_text())

NUM = [] # 노래 고유넘버
for i in range(1,101):
    number = bs.select_one('#frm > div > table > tbody > tr:nth-child({})'.format(i)).attrs['data-song-no']
    NUM.append(number)
    
    
LYRIC = [] # 노래 가사
for i in NUM:
    response_2 = requests.get('https://www.melon.com/song/detail.htm?songId=' + str(i), headers = hdr)
    html_2 = response_2.text 
    bs_2 = BeautifulSoup(html_2, 'html.parser')
    lyric = bs_2.find_all('div', {'class':'lyric'})
    
    for i in lyric:
        LYRIC.append(i.get_text().strip())

rank = list(range(1,101)) #순위   

### **1-4. 파일로 저장하기**

In [4]:
df = pd.DataFrame({'Rank':rank, 'Title':TITLE, 'Artist':ARTIST, 'Lyric':LYRIC})
df.set_index('Rank', inplace=True) # 인덱스를 '순위'로 지정
df.to_excel("melon_chart100.xlsx",  encoding='utf-8')

In [5]:
df.head()

,Title,Artist,Lyric
Rank,,,
1,라일락,아이유,나리는 꽃가루에 눈이 따끔해 (아야)눈물이 고여도 꾹 참을래내 마음 한켠 비밀스런 ...
2,롤린 (Rollin'),브레이브걸스,그 날을 잊지 못해 babe날 보며 환히 웃던 너의 미소에홀린 듯 I'm fall ...
3,Celebrity,아이유,"세상의 모서리구부정하게 커버린골칫거리 outsider걸음걸이, 옷차림, 이어폰 너머..."
4,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,I got my peaches out in GeorgiaOh yeah shitI g...
5,운전만해 (We Ride),브레이브걸스,넌 운전만 해계속 운전만 해왜 이리 된 걸까 우리 사이가갑자기 어색해졌단 걸왜 달라...


## **2. Selenium을 이용한 크롤링**

- 셀레니움(Selenium)은 웹드라이버의 API를 통해 브라우저를 제어하기 때문에, 자바스크립트에 의해 동적으로 생성되는 웹페이지의 데이터를 크롤링 할 때 매우 유용하게 사용되는 크롤링 도구이다.

- 셀레니움을 사용할 때 가장 보편적인 Chrome driver를 사용한다. 다음 링크에서 다운받을 수 있다.
https://chromedriver.chromium.org/downloads

- 셀레니움은 웹드라이버의 DOM에서 요소를 찾을 때 다양한 선택자들 중 골라서 사용하면 된다. 아래와 같이 find_element로 시작하는 함수는 조건에 맞는 요소를 하나만 반환한다. 조건을 만족하는 모든 요소를 반환받고 싶은 경우엔 find_elements로 시작하면 된다.

      *   find_element()
      *   find_element_by_css_selector()
      *   find_element_by_xpath()
      *   find_element_by_class_name()
      *   find_element_by_tag_name()
      *   find_element_by_name()
      *   find_element_by_id()
      *   find_element_by_link_text()
      *   find_element_by_partial_link_text()



### **2-1. 필요한 라이브러리 불러오기**

In [6]:
from selenium import webdriver
import pandas as pd

### **2-2. 웹 브라우저 실행하고 원하는 데이터 가져오기**

In [8]:
# 웹 브라우저 실행하기 
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('headless')
driver = webdriver.Chrome(executable_path="/Users/jinsol/Desktop/chromedriver",chrome_options=chrome_options) #webdriver 객체 생성
driver.get("https://www.melon.com/chart/index.htm")


# 원하는 데이터 가져오기
TITLE = [] # 노래 제목
title = driver.find_elements_by_class_name('ellipsis.rank01') 
for i in title:
    TITLE.append(i.text)

ARTIST = [] # 아티스트명
artist = driver.find_elements_by_class_name('ellipsis.rank02') 
for i in artist:
    ARTIST.append(i.text)
    
NUM = [] # 노래 고유넘버
for i in range(1,101):
    number = driver.find_element_by_css_selector('#frm > div > table > tbody > tr:nth-child({})'.format(i)).get_attribute('data-song-no')
    NUM.append(number)  

LYRIC=[] # 노래 가사
for i in NUM:
    driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
    lyric = driver.find_element_by_class_name("lyric")
    LYRIC.append(lyric.text)

    
rank = list(range(1,101)) #순위   

/Users/jinsol/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


### **2-3. 파일로 저장하기**

In [9]:
df = pd.DataFrame({"Rank": rank, "Title":TITLE, "Artist":ARTIST, "Lyric":LYRIC})
df.set_index('Rank', inplace=True) # 인덱스를 '순위'로 지정
df.to_excel("melon_chart100.xlsx",  encoding='utf-8')

In [10]:
df.head()

,Title,Artist,Lyric
Rank,,,
1,라일락,아이유,나리는 꽃가루에 눈이 따끔해 (아야)\n눈물이 고여도 꾹 참을래\n내 마음 한켠 비...
2,롤린 (Rollin'),브레이브걸스,그 날을 잊지 못해 babe\n날 보며 환히 웃던\n너의 미소에\n홀린 듯\nI'm...
3,Celebrity,아이유,"세상의 모서리\n구부정하게 커버린\n골칫거리 outsider\n\n걸음걸이, 옷차림..."
4,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,I got my peaches out in Georgia\nOh yeah shit\...
5,운전만해 (We Ride),브레이브걸스,넌 운전만 해\n계속 운전만 해\n왜 이리 된 걸까 우리 사이가\n갑자기 어색해졌단...
